# **Data Extraction**
For our project, we used various datasets from different sources. This includes extracting data directly from Spotify using their API, and also using Kaggle datasets.

This notebook will take us through the process of forming each dataset file. The various dataset files are as follows:
> 1. dataset.csv (retrieved directly from Spotify using API)
> 2. time.csv (modified Kaggle dataset of songs from different eras)
> 3. merged20k.csv (combination of time.csv and dataset.csv)
> 4. merged200k.csv (combination of merged140k.csv and Kaggle dataset containing 120k songs)

For the final analyses, only <u>***merged20k.csv and merged200k.csv***</u> are used.

#### **Import Necessary Libraries**
- *spotipy* is vital for the extraction of any data from Spotify
- *SpotifyClientCredentials* is necessary to obtain authorisation from Spotify


In [1]:
import spotipy
import pandas as pd
import csv
from spotipy.oauth2 import SpotifyClientCredentials
import time # time.sleep() used throughout data extraction to prevent MaxRetriesError from the API

"""
client_id and client_secret of the developer accessing the API should be written below. This field is left blank for security reasons
"""
#client_id = personal client id
#client_secret = personal client secret

# Obtain authorisation from Spotify
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, retries=0) 

---

# <u>**Section 1: Original Base Dataset -- 'dataset.csv'**</u>
In order to form our base dataset, we made our own playlist on Spotify consisting of songs from different eras, different genres, different languages, etc. This was done through the merging of multiple of Spotify's playlists.

After this [dataset playlist](https://open.spotify.com/playlist/1aA8TSi48YqaGdXNDqGrVV?si=5b086c85d48248b6) was made, we used Spotify's Web API 'Spotipy' to extract detailed information about each track (e.g. audio features, track popularity, artist popularity, etc.). The final base dataset consists of 9942 tracks.

**'dataset.csv'** is this original base dataset formed from extracting track data directly from Spotify using the API.

This section will take you through the process of track data extraction for our <u>***'dataset.csv'***</u> dataset. :
> 1. Extract Playlist Tracks
> 2. Extract Audio Features and Artist Information for each track
> 3. Write to CSV file

## **Initialise Arrays and Dataframe**

In [2]:
dataset_URI = 'spotify:playlist:1aA8TSi48YqaGdXNDqGrVV' # unique URI of our dataset playlist
track_id = []
artist_id = []
track_names = []
album_names = []
artist_genres = []
artist_pop = []
track_pop = []
album_date = []
audio_features_artists = []

In [3]:
# column names for audio features, artist information, track URI, artist URI
col_names = ['Danceability', 'Energy', 'Key', 'Loudness', 'Mode', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo', 'Duration', 'Time Signature',
             'Artist Genres', 'Artist Name', 'Artist Popularity', 'Track URI', 'Artist URI']

# create an empty dataframe with the above column names
all_tracks = pd.DataFrame(columns=col_names)

## **Helper Functions**

### **Extract Playlist Tracks**

In [4]:
def get_playlist_tracks(pl_URI, lim, offs):
    """
    The get_playlist_tracks() function extracts information about each track from the dataset playlist

    For tracks with multiple artists, the main (i.e. the first) artist was chosen as the metric of popularity
    """

    # iterate through playlist tracks
    for track in sp.playlist_tracks(pl_URI, limit=lim, offset=offs)["items"]:
        # only take in non-local tracks (i.e. tracks already on spotify)
        if not track['is_local']:
            ## URI
            track_uri = track["track"]["uri"] # get track's URI
            
            ## Track Information
            # Track name
            track_name = track["track"]["name"] # get track's name
            # Popularity of the track
            t_pop = track["track"]["popularity"] # get track's popularity index
            
            ## Artist Information
            artist_uri = track["track"]["artists"][0]["uri"] # get artists' URI
            
            ## Album Information
            # Album name
            album = track["track"]["album"]["name"] # get album name
            # Album release date in format 'YYYY-MM'
            date = track["track"]["album"]["release_date"] # get album release date

            track_id.append(track_uri)
            artist_id.append(artist_uri)
            track_names.append(track_name)
            album_names.append(album)
            track_pop.append(t_pop)
            album_date.append(date)
    

### **Get audio features and artist information of the tracks**

In [5]:
def get_audio_artist(tracks, artists, df):
    """
    Retrieves audio features, artist name, artist genres, artist popularity of the tracks in intervals of 50 tracks.

    sp.artists() is used to extract artist information. Only artist name, artist genres, artist popularity
    are extracted from the dictionary of artist information.

    sp.audio_features() is used to extract audio features. Only the audio features are extracted
    with the other track information filtered out.

    Returns dataframe containing all the above columns.
    """

    for i in range(0, len(tracks), 50):
        if (i+50) > len(tracks):
            track_interval = tracks[i:len(tracks)]
            artist_interval = artists[i:len(artists)]
        else:
            track_interval = tracks[i:i+50]
            artist_interval = artists[i:i+50]
        
        ## Retrieve artist information (i.e. popularity, genres, name)
        # Get artist object 
        artist_info = sp.artists(artist_interval) # returns an artist object corresponding to the given URI, this object contains detailed info on the artist
        # Name, popularity, genre of Artist
        artist_info = [[v for k, v in d.items() if k in ['name', 'genres', 'popularity']] for d in artist_info["artists"]]
        for a in artist_info:
            a[0] = ", ".join(a[0]) # get artists' list of genres and converts the list into a string with a comma as the separator

        ## Retrieve audio features
        audio_features = sp.audio_features(track_interval)
        audio_features = [[v for k, v in d.items() if k not in ['type', 'id', 'uri', 'track_href', 'analysis_url']] for d in audio_features]
        
        ## Combine retrieved information into one list (i.e. a row in the dataframe)
        audio_artist = [] # store all rows
        batch = i
        for j in range(len(audio_features)):
            row = audio_features[j]+ artist_info[j] + [tracks[batch]] + [artists[batch]]
            audio_artist.append(row)
            batch += 1

        print(f"{i+50} tracks done", end="; ")

        ## Add audio_artist rows into the dataframe
        audio_artist = pd.DataFrame(data=audio_artist, columns=col_names)
        df = pd.concat([df, audio_artist], join="outer", ignore_index=True)

    return df

## **Extract data using API**
First, all the tracks are extracted from the playlist using the get_playlist_tracks() function

In [6]:
for i in range(0,9901,100):
    get_playlist_tracks(dataset_URI, 100, i)
    print(f"{i+100} done", end="; ")

    if (i+100) % 1000 == 0:
        time.sleep(5)
        print()

100 done; 200 done; 300 done; 400 done; 500 done; 600 done; 700 done; 800 done; 900 done; 1000 done; 
1100 done; 1200 done; 1300 done; 1400 done; 1500 done; 1600 done; 1700 done; 1800 done; 1900 done; 2000 done; 
2100 done; 2200 done; 2300 done; 2400 done; 2500 done; 2600 done; 2700 done; 2800 done; 2900 done; 3000 done; 
3100 done; 3200 done; 3300 done; 3400 done; 3500 done; 3600 done; 3700 done; 3800 done; 3900 done; 4000 done; 
4100 done; 4200 done; 4300 done; 4400 done; 4500 done; 4600 done; 4700 done; 4800 done; 4900 done; 5000 done; 
5100 done; 5200 done; 5300 done; 5400 done; 5500 done; 5600 done; 5700 done; 5800 done; 5900 done; 6000 done; 
6100 done; 6200 done; 6300 done; 6400 done; 6500 done; 6600 done; 6700 done; 6800 done; 6900 done; 7000 done; 
7100 done; 7200 done; 7300 done; 7400 done; 7500 done; 7600 done; 7700 done; 7800 done; 7900 done; 8000 done; 
8100 done; 8200 done; 8300 done; 8400 done; 8500 done; 8600 done; 8700 done; 8800 done; 8900 done; 9000 done; 
9100 done;

Next, the audio features and artist information of the tracks are retrieved from Spotify using the API. This is done by the get_audio_artist() function

In [7]:
for i in range(0, 9901, 1000):
    print(f"Retrieving information for tracks {i} to {i+1000}")
    all_tracks = get_audio_artist(track_id[i:i+1000], artist_id[i:i+1000], all_tracks)
    print()
    time.sleep(5)

Retrieving information for tracks 0 to 1000


50 tracks done; 100 tracks done; 150 tracks done; 200 tracks done; 250 tracks done; 300 tracks done; 350 tracks done; 400 tracks done; 450 tracks done; 500 tracks done; 550 tracks done; 600 tracks done; 650 tracks done; 700 tracks done; 750 tracks done; 800 tracks done; 850 tracks done; 900 tracks done; 950 tracks done; 1000 tracks done; 
Retrieving information for tracks 1000 to 2000
50 tracks done; 100 tracks done; 150 tracks done; 200 tracks done; 250 tracks done; 300 tracks done; 350 tracks done; 400 tracks done; 450 tracks done; 500 tracks done; 550 tracks done; 600 tracks done; 650 tracks done; 700 tracks done; 750 tracks done; 800 tracks done; 850 tracks done; 900 tracks done; 950 tracks done; 1000 tracks done; 
Retrieving information for tracks 2000 to 3000
50 tracks done; 100 tracks done; 150 tracks done; 200 tracks done; 250 tracks done; 300 tracks done; 350 tracks done; 400 tracks done; 450 tracks done; 500 tracks done; 550 tracks done; 600 tracks done; 650 tracks done; 700 

## **Write to CSV File**

In [8]:
all_tracks['Track Name'] = track_names
all_tracks['Album'] = album_names
all_tracks['Album Release Date'] = album_date
all_tracks['Track Popularity'] = track_pop

# Reorder columns
all_tracks = all_tracks[['Track Name', 'Artist Name', 'Album', 'Album Release Date', 'Artist Genres',
                         'Danceability', 'Energy', 'Key', 'Loudness', 'Mode', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo', 'Duration', 'Time Signature',
                         'Artist Popularity', 'Track Popularity', 'Track URI', 'Artist URI']]

In [9]:
all_tracks

,Track Name,Artist Name,Album,Album Release Date,Artist Genres,Danceability,Energy,Key,Loudness,Mode,...,Instrumentalness,Liveness,Valence,Tempo,Duration,Time Signature,Artist Popularity,Track Popularity,Track URI,Artist URI
0,act ii: date @ 8 (feat. Drake) - remix,4batz,act ii: date @ 8 (feat. Drake) [remix],2024-03-08,,0.550,0.545,0,-9.304,1,...,0.000000,0.0608,0.412,102.469,231339,4,74,87,spotify:track:3QS9ZCtoSCJhmaJ7QNXSAS,spotify:artist:5fi7lIgJGH3RgUpJYcffQ7
1,LEAP!,Souly Had,LEAP!,2020-10-02,,0.892,0.585,11,-7.269,1,...,0.000004,0.0881,0.244,104.011,205385,4,49,38,spotify:track:3vl2uH4D2QItFFhzz5L06g,spotify:artist:20WmX0l9JgOpehDOAXY6iO
2,Overdrive,Conan Gray,Overdrive,2021-02-19,"bedroom pop, pop, pov: indie",0.568,0.788,5,-6.280,1,...,0.000000,0.1410,0.460,104.959,183070,4,81,66,spotify:track:3QyoC6OvQUmpQwQZ18iaTs,spotify:artist:4Uc8Dsxct0oMqx0P6i60ea
3,Raise y_our glass,HUH YUNJIN,Raise y_our glass,2022-08-09,,0.567,0.503,8,-6.119,1,...,0.000000,0.3690,0.184,72.798,212164,4,69,60,spotify:track:7j1vFohEm1lbsctYY9gyGd,spotify:artist:13yWtUnz63q5VIs5SwoMhy
4,In My Mind,Lyn Lapid,In My Mind,2021-08-20,gen z singer-songwriter,0.668,0.392,2,-12.178,0,...,0.002750,0.3450,0.323,135.936,162794,3,62,73,spotify:track:2W779g2UrRZtVdf3FcEOO1,spotify:artist:4pfy05cNNTacuOQ6SiSu4v
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9937,Tenerife Sea,Ed Sheeran,x (Deluxe Edition),2014-06-21,"pop, singer-songwriter pop, uk pop",0.530,0.346,8,-10.497,1,...,0.000011,0.1050,0.359,121.876,241347,4,87,68,spotify:track:1HbcclMpw0q2WDWpdGCKdS,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V
9938,Bloodstream,Ed Sheeran,x (Deluxe Edition),2014-06-21,"pop, singer-songwriter pop, uk pop",0.660,0.316,3,-11.567,0,...,0.000294,0.1040,0.543,91.207,300253,4,87,65,spotify:track:5v4sZRuvWDcisoOk1PFv6T,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V
9939,Afterglow,Ed Sheeran,Afterglow,2020-12-21,"pop, singer-songwriter pop, uk pop",0.641,0.324,11,-5.851,1,...,0.000000,0.3280,0.273,110.184,185487,4,87,71,spotify:track:0E4Y1XIbs8GrAT1YqVy6dq,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V
9940,"You Need Me, I Don't Need You",Ed Sheeran,+,2011-09-09,"pop, singer-songwriter pop, uk pop",0.749,0.837,7,-8.837,1,...,0.000000,0.5250,0.927,103.483,220413,4,87,61,spotify:track:5KL4iZkCTZyXl7KnHgfVDj,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V


In [10]:
all_tracks.to_csv('datasets/dataset.csv', index=True) # write to 'dataset.csv' in the folder 'datasets'

---

# <u>**Section 2: Time Comparison Dataset -- 'time.csv'**</u>
In order to conduct a clearer analysis of spotify tracks across time, we used a [Kaggle dataset](https://www.kaggle.com/datasets/joebeachcapital/top-10000-spotify-songs-1960-now?resource=download) that was specifically designed to <u>compromise tracks from a wide range of eras</u>.

However, this dataset did not contain the variable <u>'artist popularity'</u> which we required for analysis. It was thus necessary to use the Spotify API to extract the artist popularity values of these tracks and add them to the dataset.

<u>***'time.csv'***</u> is the Kaggle dataset with the addition of the 'artist popularity' variable.

This section will take you through the process of appending 'artist popularity' to the Kaggle dataset.
> 1. Retrieve identifying information from Kaggle dataset
> 2. Extract Artist Popularity for each track
> 3. Write to CSV file

## **Retrieve Artist URI and Track URI from CSV file**
> Artist URI is necessary to retrieve the 'artist popularity' metric\
> Track URI is necessary to <u>_filter out local files/tracks_</u> that do not exist in the Spotify database

In [11]:
timekaggle = pd.read_csv('datasets/top_10000_1960-now.csv') # retrieve Kaggle dataset
old_size = timekaggle.shape

In [12]:
artist_id = []
with open('datasets/top_10000_1960-now.csv', 'r') as file:
    file = csv.reader(file)
    header = next(file)

    """
    For tracks with multiple artists, the main (i.e. the first) artist was chosen as the metric of popularity
    """

    for line in file:
        if 'local' not in line[0]: # filter out local tracks
            line[2] = line[2].split(',')[0] # only use the main artist
            artist_id.append(line[2])
        else:
            timekaggle.drop(timekaggle.loc[timekaggle['Track URI'] == line[0]].index, inplace=True) # drop local tracks from Kaggle dataset

In [13]:
new_size = timekaggle.shape
print(f"Original Size: {old_size}")
print(f"After removing local tracks: {new_size}")

Original Size: (9999, 35)
After removing local tracks: (9997, 35)


## **Helper Function to retrieve 'artist popularity'**

In [14]:
def get_artist_pop(artists, pop_list):
     
    for i in range(0, len(artists), 50):
        if (i+50) > len(artists):
            artist_interval = artists[i:len(artists)]
        else:
            artist_interval = artists[i:i+50]

        ## Retrieve artist information (i.e. popularity, genres, name)
        # Get artist object
        artist_info = sp.artists(artist_interval) # returns an artist object corresponding to the given URI, this object contains detailed info on the artist
        
        # Retrieve artist popularity
        popularity = []
        for d in artist_info["artists"]:
            for k, v in d.items():
                if k == 'popularity':
                    popularity.append(v)

        pop_list.extend(popularity)

        print(f"{i+50} tracks done", end="; ")

## **Extract data using API**

In [15]:
artist_pop = []

for i in range(0, 9901, 1000):
    print(f"Retrieving artist popularity for tracks {i} to {i+1000}")
    get_artist_pop(artist_id[i:i+1000], artist_pop)
    print()
    time.sleep(5)

Retrieving artist popularity for tracks 0 to 1000


50 tracks done; 100 tracks done; 150 tracks done; 200 tracks done; 250 tracks done; 300 tracks done; 350 tracks done; 400 tracks done; 450 tracks done; 500 tracks done; 550 tracks done; 600 tracks done; 650 tracks done; 700 tracks done; 750 tracks done; 800 tracks done; 850 tracks done; 900 tracks done; 950 tracks done; 1000 tracks done; 
Retrieving artist popularity for tracks 1000 to 2000
50 tracks done; 100 tracks done; 150 tracks done; 200 tracks done; 250 tracks done; 300 tracks done; 350 tracks done; 400 tracks done; 450 tracks done; 500 tracks done; 550 tracks done; 600 tracks done; 650 tracks done; 700 tracks done; 750 tracks done; 800 tracks done; 850 tracks done; 900 tracks done; 950 tracks done; 1000 tracks done; 
Retrieving artist popularity for tracks 2000 to 3000
50 tracks done; 100 tracks done; 150 tracks done; 200 tracks done; 250 tracks done; 300 tracks done; 350 tracks done; 400 tracks done; 450 tracks done; 500 tracks done; 550 tracks done; 600 tracks done; 650 track

## **Write to new CSV file**

In [16]:
timekaggle["Artist Popularity"] = artist_pop # add artist popularity column
timekaggle

,Track URI,Track Name,Artist URI(s),Artist Name(s),Album URI,Album Name,Album Artist URI(s),Album Artist Name(s),Album Release Date,Album Image URL,...,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature,Album Genres,Label,Copyrights,Artist Popularity
0,spotify:track:1XAZlnVtthcDZt2NI1Dtxo,Justified & Ancient - Stand by the Jams,spotify:artist:6dYrdRlNZSKaVxYg5IrvCH,The KLF,spotify:album:4MC0ZjNtVP1nDD5lsLxFjc,Songs Collection,spotify:artist:6dYrdRlNZSKaVxYg5IrvCH,The KLF,1992-08-03,https://i.scdn.co/image/ab67616d0000b27355346b...,...,0.015800,0.112000,0.4080,0.504,111.458,4.0,NaN,Jams Communications,"C 1992 Copyright Control, P 1992 Jams Communic...",42
1,spotify:track:6a8GbQIlV8HBUW3c6Uk9PH,I Know You Want Me (Calle Ocho),spotify:artist:0TnOYISbd1XYRBk9myaseg,Pitbull,spotify:album:5xLAcbvbSAlRtPXnKkggXA,Pitbull Starring In Rebelution,spotify:artist:0TnOYISbd1XYRBk9myaseg,Pitbull,2009-10-23,https://i.scdn.co/image/ab67616d0000b27326d73a...,...,0.014200,0.000021,0.2370,0.800,127.045,4.0,NaN,Mr.305/Polo Grounds Music/J Records,"P (P) 2009 RCA/JIVE Label Group, a unit of Son...",84
2,spotify:track:70XtWbcVZcpaOddJftMcVi,From the Bottom of My Broken Heart,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Britney Spears,spotify:album:3WNxdumkSMGMJRhEgK80qx,...Baby One More Time (Digital Deluxe Version),spotify:artist:26dSoYclwsYLMAKD3tpOr4,Britney Spears,1999-01-12,https://i.scdn.co/image/ab67616d0000b2738e4986...,...,0.560000,0.000001,0.3380,0.706,74.981,4.0,NaN,Jive,P (P) 1999 Zomba Recording LLC,81
3,spotify:track:1NXUWyPJk5kO6DQJ5t7bDu,Apeman - 2014 Remastered Version,spotify:artist:1SQRv42e4PjEYfPhS0Tk9E,The Kinks,spotify:album:6lL6HugNEN4Vlc8sj0Zcse,"Lola vs. Powerman and the Moneygoround, Pt. On...",spotify:artist:1SQRv42e4PjEYfPhS0Tk9E,The Kinks,2014-10-20,https://i.scdn.co/image/ab67616d0000b2731e7c53...,...,0.568000,0.000051,0.0384,0.833,75.311,4.0,NaN,Sanctuary Records,"C © 2014 Sanctuary Records Group Ltd., a BMG C...",65
4,spotify:track:72WZtWs6V7uu3aMgMmEkYe,You Can't Always Get What You Want,spotify:artist:22bE4uQ6baNwSHPVcDxLCe,The Rolling Stones,spotify:album:0c78nsgqX6VfniSNWIxwoD,Let It Bleed,spotify:artist:22bE4uQ6baNwSHPVcDxLCe,The Rolling Stones,1969-12-05,https://i.scdn.co/image/ab67616d0000b27373d927...,...,0.675000,0.000073,0.2890,0.497,85.818,4.0,NaN,Universal Music Group,"C © 2002 ABKCO Music & Records Inc., P ℗ 2002 ...",80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,spotify:track:3kcKlOkQQEPVwxwljbGJ5p,Kernkraft 400 (A Better Day),"spotify:artist:0u6GtibW46tFX7koQ6uNJZ, spotify...","Topic, A7S",spotify:album:2NIChqkijGw4r4Dqfmg0A3,Kernkraft 400 (A Better Day),"spotify:artist:0u6GtibW46tFX7koQ6uNJZ, spotify...","Topic, A7S",2022-06-17,https://i.scdn.co/image/ab67616d0000b273e1cafe...,...,0.184000,0.000020,0.3090,0.400,125.975,4.0,NaN,Virgin,"C © 2022 Topic, under exclusive license to Uni...",73
9995,spotify:track:5k9QrzJFDAp5cXVdzAi02f,Never Say Never - Radio Edit,spotify:artist:1ScZSjoYAihNNm9qlhzDnL,Vandalism,spotify:album:2n506u3HKN3CaEDvAjv5Ct,Never Say Never,spotify:artist:1ScZSjoYAihNNm9qlhzDnL,Vandalism,2005-10-24,https://i.scdn.co/image/ab67616d0000b273b65ad4...,...,0.000354,0.011200,0.3380,0.767,130.978,4.0,NaN,Vicious,"C 2005 Vicious, a division of Vicious Recordin...",22
9996,spotify:track:5ydeCNaWDmFbu4zl0roPAH,Groovejet (If This Ain't Love) [feat. Sophie E...,"spotify:artist:4bmymFwDu9zLCiTRUmrewb, spotify...","Spiller, Sophie Ellis-Bextor",spotify:album:20Q3pGpYiyicF32x5L8ppH,Groovejet (If This Ain't Love) [feat. Sophie E...,spotify:artist:4bmymFwDu9zLCiTRUmrewb,Spiller,2000-08-14,https://i.scdn.co/image/ab67616d0000b27342781a...,...,0.000132,0.088900,0.3610,0.626,123.037,4.0,NaN,Defected Records,"C © 2021 Defected Records Limited, P ℗ 2021 De...",44
9997,spotify:track:0zKbDrEXKpnExhGQRe9dxt,Lay Low,spotify:artist:2o5jDhtHVPhrJdv3cEQ99Z,Tiësto,spotify:album:0EYKSXXTsON8ZA95BuCoXn,Lay Low,spotify:artist:2o5jDhtHVPhrJdv3cEQ99Z,Tiësto,2023

In [17]:
timekaggle.to_csv('datasets/time.csv', index=True) # write to 'time.csv'

---

# **Section 3: Merged 20k Dataset -- 'merged20k.csv'**
In order to have a larger dataset to work with for some of our analyses, we also merged our 'dataset.csv' and 'time_comparison.csv' datasets to form a bigger dataset of around 20k songs.

This dataset was named <u>***merged20k.csv***</u>. This section will take you through the process of merging the two smaller datasets.
> 1. Clean each dataset
> 2. Merge datasets
> 3. Convert Album Release Date into Album Release Decade
> 4. Write to CSV file

## **Clean datasets before merging**
Since this merged dataset is used for only some of our analyses, it is not necessary to have all the columns in the merged dataset. The datasets were cleaned by <u>filtering out unnecessary columns</u> before merging.

In [24]:
data1 = pd.read_csv("datasets/dataset.csv") 
data2 = pd.read_csv("datasets/time.csv")

First, we clean data1:

In [25]:
data1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9942 entries, 0 to 9941
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          9942 non-null   int64  
 1   Track Name          9942 non-null   object 
 2   Artist Name         9942 non-null   object 
 3   Album               9942 non-null   object 
 4   Album Release Date  9942 non-null   object 
 5   Artist Genres       9464 non-null   object 
 6   Danceability        9942 non-null   float64
 7   Energy              9942 non-null   float64
 8   Key                 9942 non-null   int64  
 9   Loudness            9942 non-null   float64
 10  Mode                9942 non-null   int64  
 11  Speechiness         9942 non-null   float64
 12  Acousticness        9942 non-null   float64
 13  Instrumentalness    9942 non-null   float64
 14  Liveness            9942 non-null   float64
 15  Valence             9942 non-null   float64
 16  Tempo 

In [26]:
cleaned_data1 = data1[["Track Name", "Danceability", "Energy", "Key", "Loudness", "Mode", "Speechiness",
                       "Acousticness", "Instrumentalness", "Liveness", "Valence", "Tempo", "Time Signature",
                       "Track URI", "Track Popularity", "Artist Popularity", "Artist URI", "Album Release Date", "Artist Genres"]]
cleaned_data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9942 entries, 0 to 9941
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Track Name          9942 non-null   object 
 1   Danceability        9942 non-null   float64
 2   Energy              9942 non-null   float64
 3   Key                 9942 non-null   int64  
 4   Loudness            9942 non-null   float64
 5   Mode                9942 non-null   int64  
 6   Speechiness         9942 non-null   float64
 7   Acousticness        9942 non-null   float64
 8   Instrumentalness    9942 non-null   float64
 9   Liveness            9942 non-null   float64
 10  Valence             9942 non-null   float64
 11  Tempo               9942 non-null   float64
 12  Time Signature      9942 non-null   int64  
 13  Track URI           9942 non-null   object 
 14  Track Popularity    9942 non-null   int64  
 15  Artist Popularity   9942 non-null   int64  
 16  Artist

After cleaning data1, the number of columns have dropped from 23 to 19. Now we move on to cleaning data2:

In [27]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9997 entries, 0 to 9996
Data columns (total 37 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            9997 non-null   int64  
 1   Track URI             9997 non-null   object 
 2   Track Name            9996 non-null   object 
 3   Artist URI(s)         9997 non-null   object 
 4   Artist Name(s)        9996 non-null   object 
 5   Album URI             9997 non-null   object 
 6   Album Name            9996 non-null   object 
 7   Album Artist URI(s)   9997 non-null   object 
 8   Album Artist Name(s)  9997 non-null   object 
 9   Album Release Date    9997 non-null   object 
 10  Album Image URL       9995 non-null   object 
 11  Disc Number           9997 non-null   int64  
 12  Track Number          9997 non-null   int64  
 13  Track Duration (ms)   9997 non-null   int64  
 14  Track Preview URL     7102 non-null   object 
 15  Explicit             

In [28]:
cleaned_data2 = data2[["Track Name", "Danceability", "Energy", "Key", "Loudness", "Mode", "Speechiness",
                       "Acousticness", "Instrumentalness", "Liveness", "Valence", "Tempo", "Time Signature",
                       "Track URI", "Popularity", "Artist Popularity", "Artist URI(s)", "Album Release Date", "Artist Genres"]]

# change column name to match cleaned_data1
cleaned_data2["Track Popularity"] = cleaned_data2["Popularity"]
cleaned_data2.drop('Popularity', axis='columns', inplace=True)

#Splitting the string by commas
cleaned_data2["Artist URI"] = [i[0] for i in cleaned_data2["Artist URI(s)"].str.split(',')]
cleaned_data2.drop('Artist URI(s)', axis='columns', inplace=True)

cleaned_data2

/var/folders/fz/fh5c1mt952s8gfzlg27lyp9c0000gn/T/ipykernel_64356/1674119244.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data2["Track Popularity"] = cleaned_data2["Popularity"]
/var/folders/fz/fh5c1mt952s8gfzlg27lyp9c0000gn/T/ipykernel_64356/1674119244.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data2.drop('Popularity', axis='columns', inplace=True)
/var/folders/fz/fh5c1mt952s8gfzlg27lyp9c0000gn/T/ipykernel_64356/1674119244.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

,Track Name,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature,Track URI,Artist Popularity,Album Release Date,Artist Genres,Track Popularity,Artist URI
0,Justified & Ancient - Stand by the Jams,0.617,0.872,8.0,-12.305,1.0,0.0480,0.015800,0.112000,0.4080,0.504,111.458,4.0,spotify:track:1XAZlnVtthcDZt2NI1Dtxo,42,1992-08-03,"acid house,ambient house,big beat,hip house",0,spotify:artist:6dYrdRlNZSKaVxYg5IrvCH
1,I Know You Want Me (Calle Ocho),0.825,0.743,2.0,-5.995,1.0,0.1490,0.014200,0.000021,0.2370,0.800,127.045,4.0,spotify:track:6a8GbQIlV8HBUW3c6Uk9PH,84,2009-10-23,"dance pop,miami hip hop,pop",64,spotify:artist:0TnOYISbd1XYRBk9myaseg
2,From the Bottom of My Broken Heart,0.677,0.665,7.0,-5.171,1.0,0.0305,0.560000,0.000001,0.3380,0.706,74.981,4.0,spotify:track:70XtWbcVZcpaOddJftMcVi,81,1999-01-12,"dance pop,pop",56,spotify:artist:26dSoYclwsYLMAKD3tpOr4
3,Apeman - 2014 Remastered Version,0.683,0.728,9.0,-8.920,1.0,0.2590,0.568000,0.000051,0.0384,0.833,75.311,4.0,spotify:track:1NXUWyPJk5kO6DQJ5t7bDu,65,2014-10-20,"album rock,art rock,british invasion,classic r...",42,spotify:artist:1SQRv42e4PjEYfPhS0Tk9E
4,You Can't Always Get What You Want,0.319,0.627,0.0,-9.611,1.0,0.0687,0.675000,0.000073,0.2890,0.497,85.818,4.0,spotify:track:72WZtWs6V7uu3aMgMmEkYe,80,1969-12-05,"album rock,british invasion,classic rock,rock",0,spotify:artist:22bE4uQ6baNwSHPVcDxLCe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9992,Kernkraft 400 (A Better Day),0.623,0.727,11.0,-5.570,0.0,0.0562,0.184000,0.000020,0.3090,0.400,125.975,4.0,spotify:track:3kcKlOkQQEPVwxwljbGJ5p,73,2022-06-17,"german dance,pop dance,pop edm,uk dance,pop da...",79,spotify:artist:0u6GtibW46tFX7koQ6uNJZ
9993,Never Say Never - Radio Edit,0.720,0.841,9.0,-6.373,1.0,0.0340,0.000354,0.011200,0.3380,0.767,130.978,4.0,spotify:track:5k9QrzJFDAp5cXVdzAi02f,22,2005-10-24,"australian dance,melbourne bounce",17,spotify:artist:1ScZSjoYAihNNm9qlhzDnL
9994,Groovejet (If This Ain't Love) [feat. Sophie E...,0.719,0.806,9.0,-6.802,0.0,0.0389,0.000132,0.088900,0.3610,0.626,123.037,4.0,spotify:track:5ydeCNaWDmFbu4zl0roPAH,44,2000-08-14,"disco house,vocal house,dance pop,europop,new ...",62,spotify:artist:4bmymFwDu9zLCiTRUmrewb
9995,Lay Low,0.534,0.855,1.0,-4.923,0.0,0.1830,0.060700,0.000263,0.3460,0.420,122.060,4.0,spotify:track:0zKbDrEXKpnExhGQRe9dxt,85,2023-01-06,"big room,brostep,dutch edm,edm,house,pop dance...",87,spotify:artist:2o5jDhtHVPhrJdv3cEQ99Z


After cleaning data2, the number of columns have dropped from 37 to 19

## **Merge datasets**

In [29]:
merged = pd.concat([cleaned_data1, cleaned_data2], ignore_index=True)
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19939 entries, 0 to 19938
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Track Name          19938 non-null  object 
 1   Danceability        19939 non-null  float64
 2   Energy              19939 non-null  float64
 3   Key                 19939 non-null  float64
 4   Loudness            19939 non-null  float64
 5   Mode                19939 non-null  float64
 6   Speechiness         19939 non-null  float64
 7   Acousticness        19939 non-null  float64
 8   Instrumentalness    19939 non-null  float64
 9   Liveness            19939 non-null  float64
 10  Valence             19939 non-null  float64
 11  Tempo               19939 non-null  float64
 12  Time Signature      19939 non-null  float64
 13  Track URI           19939 non-null  object 
 14  Track Popularity    19939 non-null  int64  
 15  Artist Popularity   19939 non-null  int64  
 16  Arti

After merging the datasets, we remove the duplicate values. This is done by finding duplicate Track URIs as the URI is unique for each track.

The number of rows drops from 19939 to 18483 as seen below, showing that removal of duplicates was successful.

In [30]:
# Remove duplicates
merged.drop_duplicates(subset=["Track URI"], inplace=True, keep="first")
merged

,Track Name,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature,Track URI,Track Popularity,Artist Popularity,Artist URI,Album Release Date,Artist Genres
0,act ii: date @ 8 (feat. Drake) - remix,0.550,0.545,0.0,-9.304,1.0,0.1630,0.318000,0.000000,0.0608,0.412,102.469,4.0,spotify:track:3QS9ZCtoSCJhmaJ7QNXSAS,87,74,spotify:artist:5fi7lIgJGH3RgUpJYcffQ7,2024-03-08,NaN
1,LEAP!,0.892,0.585,11.0,-7.269,1.0,0.1020,0.146000,0.000004,0.0881,0.244,104.011,4.0,spotify:track:3vl2uH4D2QItFFhzz5L06g,38,49,spotify:artist:20WmX0l9JgOpehDOAXY6iO,2020-10-02,NaN
2,Overdrive,0.568,0.788,5.0,-6.280,1.0,0.0377,0.010100,0.000000,0.1410,0.460,104.959,4.0,spotify:track:3QyoC6OvQUmpQwQZ18iaTs,66,81,spotify:artist:4Uc8Dsxct0oMqx0P6i60ea,2021-02-19,"bedroom pop, pop, pov: indie"
3,Raise y_our glass,0.567,0.503,8.0,-6.119,1.0,0.1240,0.581000,0.000000,0.3690,0.184,72.798,4.0,spotify:track:7j1vFohEm1lbsctYY9gyGd,60,69,spotify:artist:13yWtUnz63q5VIs5SwoMhy,2022-08-09,NaN
4,In My Mind,0.668,0.392,2.0,-12.178,0.0,0.0585,0.789000,0.002750,0.3450,0.323,135.936,3.0,spotify:track:2W779g2UrRZtVdf3FcEOO1,73,62,spotify:artist:4pfy05cNNTacuOQ6SiSu4v,2021-08-20,gen z singer-songwriter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19930,Get A Life,0.710,0.952,11.0,-6.286,0.0,0.0557,0.023200,0.000121,0.2860,0.889,137.976,4.0,spotify:track:4gFL3QgCRx0o1B5KjlkCR1,22,43,spotify:artist:0zg9mF9dX2knvdTKnL22T1,2004-03-26,"big beat,breakbeat"
19933,Kernkraft 400,0.798,0.430,8.0,-7.839,0.0,0.0868,0.005500,0.901000,0.1460,0.487,140.064,4.0,spotify:track:6PUzxtIHkv346yP89NzP9X,57,48,spotify:artist:7vFpNLbCXbBFs4kFBUlkSl,2006-03-07,german techno
19935,Never Say Never - Radio Edit,0.720,0.841,9.0,-6.373,1.0,0.0340,0.000354,0.011200,0.3380,0.767,130.978,4.0,spotify:track:5k9QrzJFDAp5cXVdzAi02f,17,22,spotify:artist:1ScZSjoYAihNNm9qlhzDnL,2005-10-24,"australian dance,melbourne bounce"
19936,Groovejet (If This Ain't Love) [feat. Sophie E...,0.719,0.806,9.0,-6.802,0.0,0.0389,0.000132,0.088900,0.3610,0.626,123.037,4.0,spotify:track:5ydeCNaWDmFbu4zl0roPAH,62,44,spotify:artist:4bmymFwDu9zLCiTRUmrewb,2000-08-14,"disco house,vocal house,dance pop,europop,new ..."


Remove any possibly empty rows in the dataset. The drop in the number of rows observed below tells us that there was one empty row.

In [31]:
merged = merged.dropna(subset=['Track Name', 'Track URI'])
merged

,Track Name,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature,Track URI,Track Popularity,Artist Popularity,Artist URI,Album Release Date,Artist Genres
0,act ii: date @ 8 (feat. Drake) - remix,0.550,0.545,0.0,-9.304,1.0,0.1630,0.318000,0.000000,0.0608,0.412,102.469,4.0,spotify:track:3QS9ZCtoSCJhmaJ7QNXSAS,87,74,spotify:artist:5fi7lIgJGH3RgUpJYcffQ7,2024-03-08,NaN
1,LEAP!,0.892,0.585,11.0,-7.269,1.0,0.1020,0.146000,0.000004,0.0881,0.244,104.011,4.0,spotify:track:3vl2uH4D2QItFFhzz5L06g,38,49,spotify:artist:20WmX0l9JgOpehDOAXY6iO,2020-10-02,NaN
2,Overdrive,0.568,0.788,5.0,-6.280,1.0,0.0377,0.010100,0.000000,0.1410,0.460,104.959,4.0,spotify:track:3QyoC6OvQUmpQwQZ18iaTs,66,81,spotify:artist:4Uc8Dsxct0oMqx0P6i60ea,2021-02-19,"bedroom pop, pop, pov: indie"
3,Raise y_our glass,0.567,0.503,8.0,-6.119,1.0,0.1240,0.581000,0.000000,0.3690,0.184,72.798,4.0,spotify:track:7j1vFohEm1lbsctYY9gyGd,60,69,spotify:artist:13yWtUnz63q5VIs5SwoMhy,2022-08-09,NaN
4,In My Mind,0.668,0.392,2.0,-12.178,0.0,0.0585,0.789000,0.002750,0.3450,0.323,135.936,3.0,spotify:track:2W779g2UrRZtVdf3FcEOO1,73,62,spotify:artist:4pfy05cNNTacuOQ6SiSu4v,2021-08-20,gen z singer-songwriter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19930,Get A Life,0.710,0.952,11.0,-6.286,0.0,0.0557,0.023200,0.000121,0.2860,0.889,137.976,4.0,spotify:track:4gFL3QgCRx0o1B5KjlkCR1,22,43,spotify:artist:0zg9mF9dX2knvdTKnL22T1,2004-03-26,"big beat,breakbeat"
19933,Kernkraft 400,0.798,0.430,8.0,-7.839,0.0,0.0868,0.005500,0.901000,0.1460,0.487,140.064,4.0,spotify:track:6PUzxtIHkv346yP89NzP9X,57,48,spotify:artist:7vFpNLbCXbBFs4kFBUlkSl,2006-03-07,german techno
19935,Never Say Never - Radio Edit,0.720,0.841,9.0,-6.373,1.0,0.0340,0.000354,0.011200,0.3380,0.767,130.978,4.0,spotify:track:5k9QrzJFDAp5cXVdzAi02f,17,22,spotify:artist:1ScZSjoYAihNNm9qlhzDnL,2005-10-24,"australian dance,melbourne bounce"
19936,Groovejet (If This Ain't Love) [feat. Sophie E...,0.719,0.806,9.0,-6.802,0.0,0.0389,0.000132,0.088900,0.3610,0.626,123.037,4.0,spotify:track:5ydeCNaWDmFbu4zl0roPAH,62,44,spotify:artist:4bmymFwDu9zLCiTRUmrewb,2000-08-14,"disco house,vocal house,dance pop,europop,new ..."


## **Convert 'Album Release Date' to 'Album Release Decade'**
We felt that analysing album release decade could be more advantageous compared to analysing the specific album release date. Broader trends in music could be observed in a longer period of time like decades rather than a short period of time like years. Analysing decades also reduces the possiblities of outliers skewing values.

In [32]:
merged['Album Release Date'] = pd.to_datetime(merged['Album Release Date'], format='mixed')
merged['Release Year'] = merged['Album Release Date'].dt.year

#function to map the year to its decade
def map_year_to_decade(year):
    return f"{year - year % 10}s"

#Create a new column containing the decade of the track
merged['Decade Released'] = merged['Release Year'].apply(map_year_to_decade)

merged['Decade Released']

/var/folders/fz/fh5c1mt952s8gfzlg27lyp9c0000gn/T/ipykernel_64356/77415046.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['Album Release Date'] = pd.to_datetime(merged['Album Release Date'], format='mixed')
/var/folders/fz/fh5c1mt952s8gfzlg27lyp9c0000gn/T/ipykernel_64356/77415046.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['Release Year'] = merged['Album Release Date'].dt.year
/var/folders/fz/fh5c1mt952s8gfzlg27lyp9c0000gn/T/ipykernel_64356/77415046.py:9: SettingWithCopyWarning: 
A

0        2020s
1        2020s
2        2020s
3        2020s
4        2020s
         ...  
19930    2000s
19933    2000s
19935    2000s
19936    2000s
19938    2020s
Name: Decade Released, Length: 18482, dtype: object

In [33]:
merged

,Track Name,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,...,Tempo,Time Signature,Track URI,Track Popularity,Artist Popularity,Artist URI,Album Release Date,Artist Genres,Release Year,Decade Released
0,act ii: date @ 8 (feat. Drake) - remix,0.550,0.545,0.0,-9.304,1.0,0.1630,0.318000,0.000000,0.0608,...,102.469,4.0,spotify:track:3QS9ZCtoSCJhmaJ7QNXSAS,87,74,spotify:artist:5fi7lIgJGH3RgUpJYcffQ7,2024-03-08,NaN,2024,2020s
1,LEAP!,0.892,0.585,11.0,-7.269,1.0,0.1020,0.146000,0.000004,0.0881,...,104.011,4.0,spotify:track:3vl2uH4D2QItFFhzz5L06g,38,49,spotify:artist:20WmX0l9JgOpehDOAXY6iO,2020-10-02,NaN,2020,2020s
2,Overdrive,0.568,0.788,5.0,-6.280,1.0,0.0377,0.010100,0.000000,0.1410,...,104.959,4.0,spotify:track:3QyoC6OvQUmpQwQZ18iaTs,66,81,spotify:artist:4Uc8Dsxct0oMqx0P6i60ea,2021-02-19,"bedroom pop, pop, pov: indie",2021,2020s
3,Raise y_our glass,0.567,0.503,8.0,-6.119,1.0,0.1240,0.581000,0.000000,0.3690,...,72.798,4.0,spotify:track:7j1vFohEm1lbsctYY9gyGd,60,69,spotify:artist:13yWtUnz63q5VIs5SwoMhy,2022-08-09,NaN,2022,2020s
4,In My Mind,0.668,0.392,2.0,-12.178,0.0,0.0585,0.789000,0.002750,0.3450,...,135.936,3.0,spotify:track:2W779g2UrRZtVdf3FcEOO1,73,62,spotify:artist:4pfy05cNNTacuOQ6SiSu4v,2021-08-20,gen z singer-songwriter,2021,2020s
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19930,Get A Life,0.710,0.952,11.0,-6.286,0.0,0.0557,0.023200,0.000121,0.2860,...,137.976,4.0,spotify:track:4gFL3QgCRx0o1B5KjlkCR1,22,43,spotify:artist:0zg9mF9dX2knvdTKnL22T1,2004-03-26,"big beat,breakbeat",2004,2000s
19933,Kernkraft 400,0.798,0.430,8.0,-7.839,0.0,0.0868,0.005500,0.901000,0.1460,...,140.064,4.0,spotify:track:6PUzxtIHkv346yP89NzP9X,57,48,spotify:artist:7vFpNLbCXbBFs4kFBUlkSl,2006-03-07,german techno,2006,2000s
19935,Never Say Never - Radio Edit,0.720,0.841,9.0,-6.373,1.0,0.0340,0.000354,0.011200,0.3380,...,130.978,4.0,spotify:track:5k9QrzJFDAp5cXVdzAi02f,17,22,spotify:artist:1ScZSjoYAihNNm9qlhzDnL,2005-10-24,"australian dance,melbourne bounce",2005,2000s
19936,Groovejet (If This Ain't Love) [feat. Sophie E...,0.719,0.806,9.0,-6.802,0.0,0.0389,0.000132,0.088900,0.3610,...,123.037,4.0,spotify:track:5ydeCNaWDmFbu4zl0roPAH,62,44,spotify:artist:4bmymFwDu9zLCiTRUmrewb,2000-08-14,"disco house,vocal house,dance pop,europop,new ...",2000,2000s


## **Write to CSV File**

In [34]:
merged.to_csv("datasets/merged20k.csv", index=False)

---

# <u>**Section 4: Merged 200k Dataset &rarr; merged200k.csv**</u>
Even with the 20k dataset, some of the variables were still extremely skewed. This can be seen in the data-visualisation.ipynb notebook.

We thus decided to merge 'merged20k.csv' with another [Kaggle dataset of 190k songs](https://www.kaggle.com/datasets/zaheenhamidani/ultimate-spotify-tracks-db) in order to increase the number of data points.

This dataset was named <u>***merged200k.csv***</u>. This section will take you through the process of merging 'merged20k.csv' and the Kaggle dataset.
> 1. Align column names
> 2. Merge datasets
> 3. Write to CSV file

## **Align column names**

In [37]:
merged20k = pd.read_csv('datasets/merged20k.csv')

# Extract Kaggle dataset into another dataframe
kaggle190k = pd.read_csv("datasets/SpotifyFeatures.csv")
kaggle190k.head()

,Artist Genres,Artist Name,Track Name,track_id,Track Popularity,Acousticness,Danceability,Duration_ms,Energy,Instrumentalness,Key,Liveness,Loudness,Mode,Speechiness,Tempo,Time Signature,Valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390


If we want to merge these two datasets, we must be careful to remove double entries. The ***track_id of kaggle120k*** and the ***last few digits of Track URI of merged20k*** is the unique code for each song. 

We need to extract the last few digits of Spotify URI using regex, and store it under a heading called 'track_id' just like in api_data_two.

In [38]:
merged20k['track_id'] = merged20k['Track URI'].str.extract(r'spotify:track:(.*)')

In [39]:
# Check if track_id is successfully extracted
merged20k.head()

,Track Name,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,...,Time Signature,Track URI,Track Popularity,Artist Popularity,Artist URI,Album Release Date,Artist Genres,Release Year,Decade Released,track_id
0,act ii: date @ 8 (feat. Drake) - remix,0.550,0.545,0.0,-9.304,1.0,0.1630,0.3180,0.000000,0.0608,...,4.0,spotify:track:3QS9ZCtoSCJhmaJ7QNXSAS,87,74,spotify:artist:5fi7lIgJGH3RgUpJYcffQ7,2024-03-08,NaN,2024,2020s,3QS9ZCtoSCJhmaJ7QNXSAS
1,LEAP!,0.892,0.585,11.0,-7.269,1.0,0.1020,0.1460,0.000004,0.0881,...,4.0,spotify:track:3vl2uH4D2QItFFhzz5L06g,38,49,spotify:artist:20WmX0l9JgOpehDOAXY6iO,2020-10-02,NaN,2020,2020s,3vl2uH4D2QItFFhzz5L06g
2,Overdrive,0.568,0.788,5.0,-6.280,1.0,0.0377,0.0101,0.000000,0.1410,...,4.0,spotify:track:3QyoC6OvQUmpQwQZ18iaTs,66,81,spotify:artist:4Uc8Dsxct0oMqx0P6i60ea,2021-02-19,"bedroom pop, pop, pov: indie",2021,2020s,3QyoC6OvQUmpQwQZ18iaTs
3,Raise y_our glass,0.567,0.503,8.0,-6.119,1.0,0.1240,0.5810,0.000000,0.3690,...,4.0,spotify:track:7j1vFohEm1lbsctYY9gyGd,60,69,spotify:artist:13yWtUnz63q5VIs5SwoMhy,2022-08-09,NaN,2022,2020s,7j1vFohEm1lbsctYY9gyGd
4,In My Mind,0.668,0.392,2.0,-12.178,0.0,0.0585,0.7890,0.002750,0.3450,...,3.0,spotify:track:2W779g2UrRZtVdf3FcEOO1,73,62,spotify:artist:4pfy05cNNTacuOQ6SiSu4v,2021-08-20,gen z singer-songwriter,2021,2020s,2W779g2UrRZtVdf3FcEOO1


## **Merge datasets**
Now, we are ready to merge the two datasets. We will only merge the common columns from both datasets. If a column only exists in one of the datasets, we will ignore it.

In [40]:
common_columns = [col for col in merged20k.columns if col in kaggle190k.columns]
df = pd.concat([merged20k[common_columns], kaggle190k[common_columns]], ignore_index = True)

In [41]:
# Check our final dataframe
df.head()

,Track Name,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature,Track Popularity,Artist Genres,track_id
0,act ii: date @ 8 (feat. Drake) - remix,0.550,0.545,0.0,-9.304,1.0,0.1630,0.3180,0.000000,0.0608,0.412,102.469,4.0,87,NaN,3QS9ZCtoSCJhmaJ7QNXSAS
1,LEAP!,0.892,0.585,11.0,-7.269,1.0,0.1020,0.1460,0.000004,0.0881,0.244,104.011,4.0,38,NaN,3vl2uH4D2QItFFhzz5L06g
2,Overdrive,0.568,0.788,5.0,-6.280,1.0,0.0377,0.0101,0.000000,0.1410,0.460,104.959,4.0,66,"bedroom pop, pop, pov: indie",3QyoC6OvQUmpQwQZ18iaTs
3,Raise y_our glass,0.567,0.503,8.0,-6.119,1.0,0.1240,0.5810,0.000000,0.3690,0.184,72.798,4.0,60,NaN,7j1vFohEm1lbsctYY9gyGd
4,In My Mind,0.668,0.392,2.0,-12.178,0.0,0.0585,0.7890,0.002750,0.3450,0.323,135.936,3.0,73,gen z singer-songwriter,2W779g2UrRZtVdf3FcEOO1


In [42]:
df.shape

(251207, 16)

Let's remove the duplicate track_id entries to ensure that each song in the dataframe is unique.

In [43]:
df = df.drop_duplicates(subset=['track_id'], keep='first')

# Check if it works
df.shape

(191395, 16)

The number of row has decreased from 251207 to 191395, indicating that duplicate songs are successfully removed.

## **Write to CSV**

In [44]:
df.to_csv("datasets/merged200k.csv", index=False)